In [1]:
import numpy as np
import pytorch_forecasting
import torch
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import random
import cv2
import numpy as np
from scipy import stats
import seaborn as sns
import seaborn as sns
import pandas as pd
from dataclasses import make_dataclass
import pdb
from processdata import load_data
from processdata import TimeSeriesDataset
import models
import torch
import scipy.io
from sklearn.preprocessing import MinMaxScaler

In [23]:
# fetch video
vid = cv2.VideoCapture("Granular Brownian Motion Square.mp4")
# attempt to extract first frame
success, frame = vid.read()
# number of frames
total_frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
# create dataframe for storing particle center coordinates and frame number.
# total frame count multiplied by maximum balls per frame
df = pd.DataFrame(index=(range(total_frame_count*1000)), columns = ['x','y','r','frame'])
# set counter to frame zero
counter = 0;
# set ball counter to zero
ball_count = 0;
# max frame count is 10
max = 100000
while (success and counter < max):
  # print(counter)
  # convert frame to greyscale
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  # Blur using 3 * 3 kernel
  gray_blurred = cv2.blur(gray, (3, 3))
  # Apply Hough transform on blurred frame
  detected_circles = cv2.HoughCircles(gray_blurred,cv2.HOUGH_GRADIENT, dp = 1, minDist = 8, param1 = 50,
                                    param2 = 9, minRadius = 3, maxRadius = 5)
  # Check if circle detected
  if detected_circles is not None:
    # Round circle parameters x, y and r to integers
    detected_circles = np.uint16(np.around(detected_circles))
    for circle in detected_circles[0, :]:
       x, y, r = circle[0], circle[1], circle[2]
       df.loc[ball_count] = [x, y, r, counter]
       ball_count = ball_count + 1
       # draw circumfrence of detected circle
       # cv2.circle(frame, (np.uint16(np.around(x)), np.uint16(np.around(y))), r, (255, 0, 255), 2)
       # draw center of detected circle (for verification)
       # cv2.circle(frame, (x, y), radius = 1, color = (0, 0, 255), thickness = 2)
  else:
    print("no circles detected")
  # mark boundaries for random sensor location placewment with green box
  # xlim (10,690) and ylim (10,710)
  # cv2.line(frame, (10, 10), (690, 10), color = (0,255,0), thickness = 1)
  # cv2.line(frame, (690, 10), (690, 710), color = (0,255,0), thickness = 1)
  # cv2.line(frame, (10, 10), (10, 710), color = (0,255,0), thickness = 1)
  # cv2.line(frame, (10, 710), (690, 710), color = (0,255,0), thickness = 1)
  # show detected particles (for verification)
  # plt.imshow(frame)
  #cv2.imshow('image',frame)
  # select next frame
  success, frame = vid.read()
  # update counter
  counter = counter + 1
  if (counter == max):
      print('reached max frames')

In [111]:
# save ball locations for movie to csv for easy access
df_all_balls = df.loc[0:ball_count-1]
df_all_balls.to_csv("./particle_position_data.csv", index= False)

In [103]:
# estimate pdf at every 50th point in frame
buffer = 50
x_interval = range(0,np.int32(np.max(df_all_balls["x"])),buffer)
y_interval = range(0,np.int32(np.max(df_all_balls["y"])),buffer)
grid_length = len(x_interval)*len(y_interval)
Q = np.zeros((total_frame_count, grid_length))
for i in range(total_frame_count):
    # print(i)
    single_frame = df[df["frame"] == i]
    x = single_frame["x"].astype(float)
    y = single_frame["y"].astype(float)
    values = np.vstack((x.ravel(), y.ravel()))
    kernel = stats.gaussian_kde(values, bw_method=0.2)
    for j in x_interval:
        for k in y_interval:
            # print(int(j/buffer*len(y_interval)+k/buffer))
            # print("pdf at: ({},{}) stored at: ({},{})".format(j, k, i, int(j/buffer*len(y_interval)+k/buffer)))
            Q[i,int(j/buffer*len(y_interval)+k/buffer)] = kernel.pdf((j,k))

In [105]:
# save pdf grid measurements for as numpy array for easy access
np.save('pdf_data.npy', Q)